In [1]:
def importlist(inputtext):
    import numpy as np
    list_of_lists = []

    with open(inputtext) as f:
        for line in f:
            inner_list = line.split("\n")
            doubleInner = [liner.strip() for liner in inner_list[0].split(" ")]
            if doubleInner[-1] == '':
                doubleInner.pop(-1)
            list_of_lists.append(doubleInner)
    flag = False
    holder = []
    objectives = []
    attributes = []
    for part in list_of_lists:
        part = list(map(int,part))
        if part[0] < 0:
            # it is an obj value
            objectives.append(part)
            flag = True
            continue
        if flag:
            # it is constraints
            attributes.append(part)
            continue
        #it is the first guys
        holder.append(part[0])
    itemCount = holder[0]
    b = holder[1:]
    return itemCount, np.array(b), np.array(objectives).reshape(-1,itemCount), np.array(attributes).transpose()

In [2]:
n,b,c,a = importlist("2D2KEasyinput.txt")
import time

In [12]:
def rectangular(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
     
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0], (tempNW[0] - 1, ((R[0][1] + R[1][1])/2.0) + .001)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1]+.001 , name = 'a')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        old = m.getConstrByName('a')
        m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
    return found, numRect

In [13]:
def rectangularThirds(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
        #if R[0][0] >= R[1][0] or R[1][1] >= R[0][1]:
         #   continue
     
        #8
        delta = (R[0][1] - R[1][1])/3
        R2 = [(R[0][0] , (R[0][1]-(2*delta)))    , R[1]]
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        #m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R2[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) >= R2[0][0], name = 'left bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[1][1], name = 'bottom bound')
        
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = round(m.objVal)
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #old = m.getConstrByName('left bound')
        #m.remove(old)
        #old = m.getConstrByName('right bound')
        #m.remove(old)
        old = m.getConstrByName('bottom bound')
        m.remove(old)
        #9
        tempNW = (z1,hold)
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        if tempNW[0] == R2[0][0]:
            continue
        
        #EXCLUDE BOTTOM, INCLUDE TOP
        
        
        R3 =[    (R[0][0] , R[0][1]-delta-.001)    , (tempNW[0]-1,R2[0][1] + .001)]
        
        #NW 
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R3[1][1], name = 'bottom bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R3[0][1], name = 'top bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) >= R3[0][0], name = 'left bound')
        m.optimize()
        if m.status == 2:
            lexConstraint = round(m.objVal)
            m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
            m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
            m.optimize()
            hold = round(m.objVal)
            NW = (lexConstraint,hold)
    
        #SE 
            old = m.getConstrByName('temp')
            m.remove(old)
            m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
            m.optimize()
            lexConstraint = round(m.objVal)
            m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
            m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
            m.optimize()
            hold = round(m.objVal)
            SE = (hold, lexConstraint)
            old = m.getConstrByName('temp')
            m.remove(old)
            old = m.getConstrByName('bottom bound')
            m.remove(old)
            old = m.getConstrByName('top bound')
            m.remove(old)
            old = m.getConstrByName('right bound')
            m.remove(old)
            old = m.getConstrByName('left bound')
            m.remove(old)
            
            found.append(list(SE))
            if NW != SE:
                found.append(list(NW))
                rectangles.append([NW,SE])
            if NW[0] == R2[0][0]:
                continue
                
        else:
            NW = tempNW
            old = m.getConstrByName('bottom bound')
            m.remove(old)
            old = m.getConstrByName('top bound')
            m.remove(old)
            old = m.getConstrByName('right bound')
            m.remove(old)
            old = m.getConstrByName('left bound')
            m.remove(old)
        
        
        #13
        #KEEP tempNW[0] IN MIND TO BE CHANGED BY R3 (MIDDLE)
        R4 = [R[0],      (NW[0] - 1, R[0][1]-delta)]
        
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R4[1][1], name = 'bottom bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R4[0][1], name = 'top bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R4[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) >= R4[0][0], name = 'left bound')
        
        m.optimize()
        z1=round(m.objVal)
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('bottom bound')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('left bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 3
    return found, numRect

In [14]:
def rectangularRand(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
        
        cut = np.random.random()
        mid = cut*(R[0][1])+(1-cut)*(R[1][1])
     
        #8
        R2 = [(R[0][0] , mid)    , R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13 
        R3 = [R[0],    (tempNW[0] - 1, mid)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1] , name = 'bottom bound')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
    return found, numRect

In [15]:
found, _ = rectangular(c,a,b)

KeyboardInterrupt: 

In [ ]:
found, len(found)

In [23]:
found, _ = rectangularThirds(c,a,b)

In [24]:
found, len(found)

([[-10608.0, -10350.0],
  [-10090.0, -10812.0],
  [-10487, -10664],
  [-10493, -10656],
  [-10578, -10512],
  [-10580, -10498],
  [-10317, -10765],
  [-10319, -10762],
  [-10419, -10715],
  [-10423, -10712],
  [-10529, -10613],
  [-10531, -10607],
  [-10558, -10562],
  [-10560, -10549],
  [-10590, -10458],
  [-10592, -10447],
  [-10598, -10414],
  [-10599, -10381],
  [-10168, -10800],
  [-10182, -10795],
  [-10272, -10781],
  [-10275, -10771],
  [-10357, -10748],
  [-10364, -10746],
  [-10391, -10731],
  [-10408, -10727],
  [-10446, -10696],
  [-10449, -10695],
  [-10475, -10682],
  [-10480, -10671],
  [-10506, -10642],
  [-10509, -10640],
  [-10514, -10636],
  [-10516, -10627],
  [-10538, -10594],
  [-10543, -10591],
  [-10545, -10576],
  [-10571, -10540],
  [-10573, -10515],
  [-10581, -10486],
  [-10584, -10480],
  [-10585, -10475],
  [-10588, -10465],
  [-10594, -10433],
  [-10602, -10371],
  [-10606, -10353],
  [-10106, -10806],
  [-10156, -10804],
  [-10208, -10792],
  [-10215, -

In [19]:
found, _ = rectangularRand(c,a,b)

In [20]:
print(found, len(found))

[[-10608.0, -10350.0], [-10090.0, -10812.0], [-10543.0, -10591.0], [-10545.0, -10577.0], [-10434.0, -10703.0], [-10437.0, -10701.0], [-10588.0, -10465.0], [-10590.0, -10458.0], [-10319.0, -10762.0], [-10335.0, -10756.0], [-10502.0, -10649.0], [-10506.0, -10642.0], [-10571.0, -10540.0], [-10573.0, -10515.0], [-10598.0, -10414.0], [-10599.0, -10381.0], [-10230.0, -10788.0], [-10237.0, -10785.0], [-10391.0, -10731.0], [-10408.0, -10727.0], [-10475.0, -10682.0], [-10480.0, -10671.0], [-10527.0, -10621.0], [-10529.0, -10613.0], [-10558.0, -10562.0], [-10560.0, -10549.0], [-10580.0, -10498.0], [-10581.0, -10486.0], [-10592.0, -10447.0], [-10594.0, -10433.0], [-10602.0, -10371.0], [-10606.0, -10353.0], [-10168.0, -10800.0], [-10182.0, -10795.0], [-10272.0, -10781.0], [-10275.0, -10771.0], [-10364.0, -10746.0], [-10366.0, -10740.0], [-10419.0, -10715.0], [-10423.0, -10712.0], [-10450.0, -10694.0], [-10452.0, -10690.0], [-10487.0, -10664.0], [-10493.0, -10657.0], [-10514.0, -10636.0], [-10516.0